<p style = "font-weight:bold; font-size:200%">paramiko ssh & scp ベース</p>

# 前処理

## 接続情報

In [ ]:
remote_host      = '192.168.0.1'
remote_user      = 'user'
remote_keyfile   = 'key.pem'
remote_password  = None

## インポート

In [ ]:
import paramiko, scp

## ロギング

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.basicConfig(
    format='[%(asctime)s] %(name)s %(levelname)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger

## 関数

### コマンドを実行
標準出力と標準エラーをまとめて返す。

In [ ]:
def exec_command(client, command, sudo_passwd=''):
    logger.debug("[command] " + command)
    output = ""
    stdin, stdout, stderr = client.exec_command(command, get_pty=True)
    if stdout.channel.closed is False:
        stdin.write(sudo_passwd + "\n")
        stdin.flush()
    for line in stdout:
        output = output + line
    for line in stderr:
        output = output + line
    return output

# 接続

## SSH接続

In [ ]:
sshclient = paramiko.SSHClient()
sshclient.load_system_host_keys()
sshclient.set_missing_host_key_policy(paramiko.AutoAddPolicy())
sshclient.connect(
    hostname=remote_host,
    username=remote_user,
    key_filename=remote_keyfile,
    password=remote_password,
    port=22,
    pkey=None,
    timeout=None,
    allow_agent=True,
    look_for_keys=True,
    compress=False,
    sock=None,
    gss_auth=False,
    gss_kex=False,
    gss_deleg_creds=True,
    gss_host=None,
    banner_timeout=None,
    auth_timeout=None,
    channel_timeout=None,
    gss_trust_dns=True,
    passphrase=None,
    disabled_algorithms=None,
    transport_factory=None,
    auth_strategy=None
)
sshclient

## SCP接続

In [ ]:
scpclient = scp.SCPClient(sshclient.get_transport(), sanitize=lambda x: x)
scpclient

# 処理実行

## コマンド実行

In [ ]:
print(exec_command(sshclient,
    'hostname'
))

In [ ]:
print(exec_command(sshclient,
    'sudo whoami',
    'sudopasswd'
))

## ファイル転送

In [ ]:
scpclient.get(
    remote_path='/pathto/file',
    local_path='./local/'
)

# 切断

In [ ]:
scpclient.close()

In [ ]:
sshclient.close()